In [1]:
import re
import httpx
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
with httpx.Client() as client:
    r = client.get("https://www.tabc.texas.gov/services/tabc-licenses-permits/tabc-license-permit-types/")
    if r.status_code == httpx.codes.OK:
        page = BeautifulSoup(r.text, 'html.parser')

In [10]:
type(page.select('div')[0])

bs4.element.Tag

In [3]:
def parse_head(head):
    head = head.strip()
    i = head.rfind("(")
    return head[(i+1):-1], head[:(i-1)]

def parse_multi_codes(code, name):
    idx = code.find("if")
    codes = code[:idx].strip().split(" or ")
    names = [
        name, 
        f"{name} ({code[idx:].strip().split(' is a ')[1].title()})"
    ]
    return zip(codes,names)

def parse_cards(cards, group):
    permit_list = list()
    for card in cards:  
        code, name = parse_head(card.select('div.card-header')[0].text)
        info = card.find('p').text.replace(u'\xa0', ' ')
        if len(code) > 2:
            for code, name in parse_multi_codes(code, name):
                permit_list.append({
                    'code': code,
                    'name': name,
                    'info': info,
                    'group': group
                })
        else:
            permit_list.append({
                    'code': code,
                    'name': name,
                    'info': info,
                    'group': group
            })
    return permit_list

        


In [4]:
permit_groups = [div.text for div in page.find("main").select("div.row")[0].find_all("h2")]
permit_types = page.find("main").select("div.row")[0].select("div.authored-accordion")
permit_list = list()
for group, accordion in zip(permit_groups, permit_types):
    permit_list.extend(parse_cards(accordion.select("div.accordion-card"), group))
df = pd.DataFrame(permit_list)

In [8]:
df.to_csv('tabc_permit_types.csv')

In [7]:
type(zip([0,1],[9,4]))

zip